In [1]:
import os
import cv2
import sys
import timm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch import nn
from tqdm import tqdm
from skimage import io
from pathlib import Path
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
# serve per ricaricare automaticamente il codice modificato
%load_ext autoreload
%autoreload 2

In [3]:
# directory principale di training, perche' nel file .csv ci sono solo i nomi dei file (immagini), da modificare per 
path = Path(os.getcwd()).parent.parent
real_data_dir = os.path.join(path, "artifact", "coco")
fake_data_dir = os.path.join(path, "artifact", "cycle_gan")

BATCH_SIZE = 32

LR = 0.001

EPOCHS = 30

DEVICE = 'cpu'

In [4]:
# df che devo creare
csv_path = os.path.join("..", "datasets", "out.csv")
df = pd.read_csv(csv_path)
df.head()
train_df, valid_df = train_test_split(df, test_size=0.20, random_state=42)

print(train_df.size)
print(valid_df.size)

2400
600


In [5]:
# carica le immagini nel dataset
class APN_Dataset(Dataset):

  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    
    A_img = io.imread(os.path.join(real_data_dir, row.Anchor))
    P_img = io.imread(os.path.join(real_data_dir, row.Positive))
    N_img = io.imread(os.path.join(fake_data_dir, row.Negative))

    # permute because the third channel has to be in first channel in torch

    A_img = torch.from_numpy(A_img).permute(2, 0, 1) / 255.0
    P_img = torch.from_numpy(P_img).permute(2, 0, 1) / 255.0
    N_img = torch.from_numpy(N_img).permute(2, 0, 1) / 255.0
    
    #A_img = np.expand_dims(A_img, 0)
    #P_img = np.expand_dims(P_img, 0)
    #N_img = np.expand_dims(N_img, 0)
    
    # normalizzazione per non far divergere il comportamento della rete
    # il valore dell'immagine sarà compreso tra 0 e 1
    #A_img = torch.from_numpy(A_img) / 255.0
    #P_img = torch.from_numpy(P_img) / 255.0
    #N_img = torch.from_numpy(N_img) / 255.0

    #A_img = torch.from_numpy(A_img.astype(np.int32)) / 65536.0
    #P_img = torch.from_numpy(P_img.astype(np.int32)) / 65536.0
    #N_img = torch.from_numpy(N_img.astype(np.int32)) / 65536.0

    return A_img, P_img, N_img

In [6]:
trainset = APN_Dataset(train_df)
validset = APN_Dataset(valid_df)

print(f"Size of trainset: {len(trainset)}")
print(f"Size of validset: {len(validset)}")

Size of trainset: 800
Size of validset: 200


In [7]:
trainloader = DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)
validloader = DataLoader(validset, batch_size = BATCH_SIZE)

In [8]:
print(f"No. of batches in trainloader : {len(trainloader)}")
print(f"No. of batches in validloader : {len(validloader)}")

No. of batches in trainloader : 25
No. of batches in validloader : 7


In [9]:
# FUNZIONE PER CARICARE IL MODELLO DI RETE NEURALE DIRETTAMENTE DALLE REPOSITORY ONLINE
class APN_Model(nn.Module):

  # QUI DEFINISCO LA SIZE DEL VETTORE DI EMBEDDING
  def __init__(self, emb_size = 512):
    super(APN_Model, self).__init__()

    # QUI CAIRCATE IL MODELLO, IN QUESTO CASO EFFICIENTNET VERSIONE B0 (LA PIù LEGGERA DELLA FAMIGLIA)
    self.efficientnet = timm.create_model('tf_efficientnetv2_b0', pretrained=False)
    self.efficientnet.classifier = nn.Linear(in_features=self.efficientnet.classifier.in_features, out_features=emb_size)

  def forward(self, images):
    embeddings = self.efficientnet(images)
    return embeddings

In [10]:
# QUI FATE UNA PICCOLA MODIFICA ALLA RETE PER FARLE AVERE IN INPUT IMMAGINI IN SCALA DI GRIGIO DELLO SPETTRO DI FOURIER
model = APN_Model()
#model.efficientnet.conv_stem = nn.Conv2d(1, 32, 3, 2, 1, bias=False)

model.to(DEVICE)

APN_Model(
  (efficientnet): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
      )
      (1): Sequential(
        (0): EdgeResidual(
          (conv_exp): Conv2dSame(16, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (bn1): BatchNormAct2d(
            64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act)

In [11]:
# funzione di train
def train_fn(model, dataloader, optimizer, criterion):
  model.train() #ON Dropout
  total_loss = 0.0

  for A, P, N in tqdm(dataloader):
    A, P, N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)

    print("debug, A shape: " + str(A.shape), file=sys.stderr)
    print("debug, P shape: " + str(P.shape), file=sys.stderr)
    print("debug, N shape: " + str(N.shape), file=sys.stderr)

    # qui vengono creati gli embeddings, le cui distanze verranno calcolate dopo
    A_embs = model(A)
    P_embs = model(P)
    N_embs = model(N)

    # criterion è la funzione di loss triplet
    loss = criterion(A_embs, P_embs, N_embs)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    return total_loss / len(dataloader)


In [12]:
# funzione di evaluation
def eval_fn(model, dataloader, criterion):
  model.eval() #OFF Dropout
  total_loss = 0.0

  with torch.no_grad():
    for A, P, N in tqdm(dataloader):
      A, P, N = A.to(DEVICE), P.to(DEVICE), N.to(DEVICE)

      A_embs = model(A)
      P_embs = model(P)
      N_embs = model(N)

      loss = criterion(A_embs, P_embs, N_embs)

      total_loss += loss.item()

    return total_loss / len(dataloader)

In [13]:
# triplet loss e adam
criterion = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

In [14]:
# training
best_valid_loss = np.Inf

for i in range(EPOCHS):
  train_loss = train_fn(model, trainloader, optimizer, criterion)
  valid_loss = eval_fn(model, validloader, criterion)

  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), 'best_model.pt')
    best_valid_loss = valid_loss
    print("SAVED_WEIGHTS_SUCCESS")

  print(f"EPOCHS : {i+1} train_loss : {train_loss} valid_loss : {valid_loss}")

  0%|          | 0/25 [00:00<?, ?it/s]debug, A shape: torch.Size([32, 3, 200, 200])
debug, P shape: torch.Size([32, 3, 200, 200])
debug, N shape: torch.Size([32, 3, 200, 200])
100%|██████████| 7/7 [00:40<00:00,  5.82s/it]


SAVED_WEIGHTS_SUCCESS
EPOCHS : 1 train_loss : 0.07709242343902588 valid_loss : 1.0001568964549474


  0%|          | 0/25 [00:00<?, ?it/s]debug, A shape: torch.Size([32, 3, 200, 200])
debug, P shape: torch.Size([32, 3, 200, 200])
debug, N shape: torch.Size([32, 3, 200, 200])
100%|██████████| 7/7 [00:12<00:00,  1.73s/it]


EPOCHS : 2 train_loss : 0.06895400047302246 valid_loss : 1.0003060443060738


  0%|          | 0/25 [00:00<?, ?it/s]debug, A shape: torch.Size([32, 3, 200, 200])
debug, P shape: torch.Size([32, 3, 200, 200])
debug, N shape: torch.Size([32, 3, 200, 200])
100%|██████████| 7/7 [00:12<00:00,  1.76s/it]


...

EPOCHS : 28 train_loss : 0.03428994655609131 valid_loss : 1.392177769115993


  0%|          | 0/25 [00:00<?, ?it/s]debug, A shape: torch.Size([32, 3, 200, 200])
debug, P shape: torch.Size([32, 3, 200, 200])
debug, N shape: torch.Size([32, 3, 200, 200])
100%|██████████| 7/7 [00:11<00:00,  1.69s/it]


EPOCHS : 29 train_loss : 0.04349386692047119 valid_loss : 1.2390277556010656


  0%|          | 0/25 [00:00<?, ?it/s]debug, A shape: torch.Size([32, 3, 200, 200])
debug, P shape: torch.Size([32, 3, 200, 200])
debug, N shape: torch.Size([32, 3, 200, 200])
100%|██████████| 7/7 [00:11<00:00,  1.68s/it]

EPOCHS : 30 train_loss : 0.039689018726348876 valid_loss : 1.4396389382226127


In [15]:
# QUESTA E' LA FUNZIONE PER GENERARE I VETTORI DI ENCODING o embeddings
def get_encoding_csv(model, anc_img_names, dirFolder):
  anc_img_names_arr = np.array(anc_img_names)
  encodings = []

  model.eval()

  with torch.no_grad():
    for i in tqdm(anc_img_names_arr):
      A = io.imread(os.path.join(dirFolder,i))
      A = torch.from_numpy(A).permute(2, 0, 1) / 255.0
      #A = np.expand_dims(A, 0)
      #A = torch.from_numpy(A.astype(np.int32)) / 255.0
      A = A.to(DEVICE)
      A_enc = model(A.unsqueeze(0))
      encodings.append(A_enc.squeeze().cpu().detach().numpy())

    encodings = np.array(encodings)
    encodings = pd.DataFrame(encodings)
    df_enc = pd.concat([anc_img_names, encodings], axis = 1)

    return df_enc

In [16]:
# QUI RICARICO IL MODELLO UNA VOLTA TRAINATO
model.load_state_dict(torch.load('best_model.pt'))

# QUI CREO IL DATABASE DI FEATURE VECTORS DEL TRAINING SET
# gli embeddings vengono aggiunti nel file csv per non rifarlo ad ogni allenamento
df_enc = get_encoding_csv(model, df['Anchor'], real_data_dir)

100%|██████████| 1000/1000 [00:53<00:00, 18.84it/s]


In [17]:
# QUI IL DATABASE COME CSV IN MODO TALE DA NON DOVER FARE QUESTA OPERAZIONE OGNI VOLTA
# OVVIAMENTE, SE DEVO FARE UN NUOVO TRAINING DEVO ANCHE RICREARE GLI ENCODINGS
df_enc.to_csv('database.csv', index = False)

df_enc = pd.read_csv('database.csv')
df_enc.head()

,Anchor,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,coco/coco2017/test2017/img001490.jpg,-0.001477,-0.042075,0.060224,-0.009346,-0.055168,-0.069095,-0.011967,0.002012,-0.000162,...,0.002250,-0.039156,0.002711,0.015507,0.090329,-0.029148,0.055215,-0.002322,0.015787,0.027064
1,coco/coco2017/train2017/img108618.jpg,-0.001517,-0.041955,0.060197,-0.009314,-0.055062,-0.068977,-0.012157,0.001882,-0.000054,...,0.002544,-0.039001,0.002708,0.015838,0.090361,-0.028957,0.055266,-0.002278,0.015625,0.026686
2,coco/coco2017/test2017/img000671.jpg,-0.001449,-0.041882,0.060361,-0.009177,-0.054836,-0.069053,-0.012113,0.001896,-0.000160,...,0.002107,-0.039140,0.002775,0.015819,0.090263,-0.029035,0.055281,-0.002281,0.015615,0.026913
3,coco/coco2017/test2017/img001694.jpg,-0.001467,-0.042130,0.060123,-0.009203,-0.055130,-0.069195,-0.012029,0.002117,-0.000144,...,0.002058,-0.039334,0.002682,0.015469,0.090379,-0.029213,0.055229,-0.002355,0.015830,0.027185
4,coco/coco2017/test2017/img030759.jpg,-0.001396,-0.042091,0.060217,-0.009294,-0.055204,-0.069004,-0.012066,0.002020,-0.000075,...,0.002389,-0.039104,0.002778,0.015493,0.090327,-0.029045,0.055220,-0.002149,0.015690,0.026920


In [18]:
# approssimazione della distanza, senza la radice quadrata, per fare i primi
# allenamenti velocemente
def euclidean_dist(img_enc, anc_enc_arr):
    #dist = np.sqrt(np.dot(img_enc-anc_enc_arr, (img_enc- anc_enc_arr).T))
    dist = np.dot(img_enc-anc_enc_arr, (img_enc- anc_enc_arr).T)
    #dist = np.sqrt(dist)
    return dist

In [19]:
df = pd.read_csv('testList.csv')

print(df['real'])
print(df.size)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'testList.csv'

In [ ]:
def getImageEmbeddings(img, model):

    img = np.expand_dims(img, 0)
    img = torch.from_numpy(img) / 255
    model.eval()

    with torch.no_grad():
        img = img.to(DEVICE)
        img_enc = model(img.unsqueeze(0))
        img_enc = img_enc.detach().cpu().numpy()
        img_enc = np.array(img_enc)

    return img_enc

In [ ]:
def searchInDatabase(img_enc, database):
    anc_enc_arr = database.iloc[:, 1:].to_numpy()
    anc_img_names = database['Anchor']

    distance = []
    for i in range(anc_enc_arr.shape[0]):
        dist = euclidean_dist(img_enc, anc_enc_arr[i : i+1, :])
        distance = np.append(distance, dist)

    closest_idx = np.argsort(distance)

    return database['Anchor'][closest_idx[0]]

In [ ]:
DataTestReal = 'C:/Users/polsi/Desktop/Lavori/DeepFake/Datasets/Artifact/cycle_gan/st/test/'
y_true = []
y_pred = []
tempDf = df
tempDf.head()
tempDf.shape

In [ ]:
# Testo i fake
currentTest = 'fake'
database = df_enc
# Prendo i primi 500 Fake
for index, row in tqdm(tempDf.iterrows()):
    img_name = DataTestReal + row[currentTest]

    img = io.imread(img_name)

    img_enc = getImageEmbeddings(img, model)

    closestLabel = searchInDatabase(img_enc, database)

    if "real" in closestLabel:
        y_pred.append("real")
    else:
        y_pred.append("fake")

In [ ]:
print(len(y_true))
print(len(y_pred))
print(y_pred)

In [ ]:
database = df_enc

In [ ]:
# Testo i real
currentTest = 'real'
# Prendo i primi 500 Fake
for index, row in tqdm(tempDf.iterrows()):
    img_name = DataTestReal + row[currentTest]
    img = io.imread(img_name)

    img_enc = getImageEmbeddings(img, model)

    closestLabel = searchInDatabase(img_enc, database)
    if "real" in closestLabel:
        y_pred.append("real")
    else:
        y_pred.append("fake")


In [ ]:
print(len(y_true))
print(len(y_pred))
print(y_pred)

In [ ]:
# Creo i vettori di ground truth
y_true = np.array(['fake'] * 1523)
print(y_true.shape)

temp = np.array(['real'] * 1523)
print(temp.shape)

y_true = np.concatenate([y_true, temp])
print(y_true.shape)

# Calcolo la matrice di confusione
confusion_matrix(y_true, y_pred, labels=["real", "fake"])

In [ ]:
# Estraggo dalla matrice di confusione i True Negative, False Positive, False Negative, True Positive
TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=["real", "fake"]).ravel()

In [ ]:
# Calcolo alcune metriche per vedere come si comporta
accuracy = round((TP + TN) /(TP + TN + FP + FN), 4) * 100
precision = round((TP) / (TP + FP), 4) * 100
sensitivy_recall = round((TP) / (TP + FN), 4) * 100
specificity = round((TN) / (TN + FP) * 100, 4)
F1_score = round((2* precision * sensitivy_recall) / (precision + sensitivy_recall), 2)

print({"Accuracy":accuracy,"Precision":precision,"Sensitivity_recall":sensitivy_recall, "Specificity": specificity, "F1_score":F1_score})